## Análise de indice de aprovação para cada disciplina

### Importando bibliotecas

In [1]:
import pandas as pd
from statistics import mean, median, mode, stdev, variance
from os import listdir
from math import modf, inf

In [2]:
import os
import sys
sys.path.append(os.path.abspath("util.py"))

import util 

### Arquivos CSV

In [3]:
discentes = util.retorna_discentes()

componentes = util.retorna_componentes()

matriculas = util.retorna_matriculas()

In [4]:
discentes.head()

,matricula,cra,periodo_ingresso,descricao,periodo_conclusao
0,18511024,8.18,1985.1,CONCLUÍDO,1988.2
1,18511029,7.70,1985.1,CONCLUÍDO,1988.2
2,18511030,9.22,1985.1,CONCLUÍDO,1988.2
3,18511033,8.31,1985.1,CONCLUÍDO,1988.2
4,18511035,8.06,1985.1,CONCLUÍDO,1988.2


In [5]:
matriculas.head()

,matricula,periodo_matricula,codigo_componente,descricao,media_final
0,10011003,2000.1,1103114,APROVADO,9.7
1,10011003,2000.1,1107137,APROVADO,10.0
2,10011003,2000.1,1103118,APROVADO,9.8
3,10011003,2000.1,1107136,APROVADO,10.0
4,10011003,2000.1,1101117,APROVADO,10.0


In [6]:
#tipos de valores para a descrição

aux = matriculas['descricao'].value_counts()

In [21]:
aux

APROVADO       29662
DISPENSADO      3610
TRANCADO        1676
REPROVADO       1310
REP. FALTA      1009
INDEFERIDO       314
EXCLUIDA         129
DESISTENCIA       26
MATRICULADO       21
CANCELADO          8
APROVEITADO        2
Name: descricao, dtype: int64

In [7]:
type(aux)

pandas.core.series.Series

In [8]:
situacoes = aux.index.to_list()

In [22]:
situacoes

['APROVADO',
 'DISPENSADO',
 'TRANCADO',
 'REPROVADO',
 'REP. FALTA',
 'INDEFERIDO',
 'EXCLUIDA',
 'DESISTENCIA',
 'MATRICULADO',
 'CANCELADO',
 'APROVEITADO']

In [9]:
componentes.head()

,codigo,nome,ch_total
0,1101101,COMPLEMENTOS EM FISICA GERAL I,30
1,1101107,ESTRUTURA DA MATÉRIA I,60
2,1101117,FISICA GERAL I,90
3,1101118,FISICA GERAL II,90
4,1101119,FISICA GERAL III,90


In [10]:
class dados_disciplina:
    
    def __init__(self, qtd_matriculas, cod, nome_componente, aprovacoes, porcentagem_aprovacao, reprovacoes, porcentagem_reprovacao, indices_str):
        self.qtd_matriculas = qtd_matriculas
        self.cod = cod
        self.nome_componente = nome_componente
        self.aprovacoes = aprovacoes
        self.porcentagem_aprovacao = porcentagem_aprovacao
        self.reprovacoes = reprovacoes
        self.porcentagem_reprovacao = porcentagem_reprovacao
        self.indices_str = indices_str
        
    def get_qtd_matriculas(self):
        return self.qtd_matriculas
    def get_cod(self):
        return self.cod
    def get_nome_componente(self):
        return self.nome_componente
    def get_aprovacoes(self):
        return self.aprovacoes
    def get_porcentagem_aprovacao(self):
        return self.porcentagem_aprovacao
    def get_reprovacoes(self):
        return self.reprovacoes
    def get_porcentagem_reprovacao(self):
        return self.porcentagem_reprovacao
    def get_indices_str(self):
        return self.indices_str

### Separando as matrículas em cada disciplina

In [11]:
cod_matriculas = {} #dicionário auxiliar com chava sendo código da disciplina e o conteúdo uma lista de dataframes com as matrículas nesse componente

for componente in componentes['codigo'].to_list():
    #print(componente)
    cod_matriculas[componente] = matriculas[matriculas['codigo_componente'] == componente]

In [12]:
len(componentes['codigo'].to_list())

291

In [13]:
teste = cod_matriculas['1101101']
teste

,matricula,periodo_matricula,codigo_componente,descricao,media_final
24069,18511022,1986.1,1101101,APROVADO,10.0
24171,18511024,1986.1,1101101,APROVADO,8.0
24224,18511026,1986.1,1101101,APROVADO,10.0
24274,18511027,1986.1,1101101,APROVADO,9.5
24328,18511029,1986.1,1101101,APROVADO,6.3
24601,18511035,1986.1,1101101,APROVADO,8.5
24650,18511036,1986.1,1101101,APROVADO,8.5
24696,18511037,1986.1,1101101,APROVADO,10.0
24795,18511040,1986.1,1101101,APROVADO,10.0


In [14]:
teste[teste['descricao'] == 'APROVADO'].count().iloc[0]

9

In [15]:
def gera_relatorio(df_componente, lista_componentes):
    #print(df_componente)
    
    qtd_matriculas = df_componente.count().iloc[0]
    cod = df_componente['codigo_componente'].iloc[0]
    nome_componente = componentes[componentes['codigo'] == cod]['nome'].iloc[0]
    aprovacoes = df_componente[df_componente['descricao'] == 'APROVADO'].count().iloc[0]
    porcentagem_aprovacao = aprovacoes/qtd_matriculas
    reprovacoes = df_componente[df_componente['descricao'] == 'REPROVADO'].count().iloc[0]
    porcentagem_reprovacao = reprovacoes/qtd_matriculas
    indices_str = ''
    
    for situacao in situacoes:
        indices_str += ('{}: {}\n'.format(situacao, df_componente[df_componente['descricao'] == situacao].count().iloc[0]))
        indices_str += ('% {}: {}\n'.format(situacao, (df_componente[df_componente['descricao'] == situacao].count().iloc[0] / qtd_matriculas)))
    
    lista_componentes.append(dados_disciplina(qtd_matriculas, cod, nome_componente, aprovacoes, porcentagem_aprovacao, reprovacoes, porcentagem_reprovacao, indices_str))

In [16]:
lista_componentes = []

for df_componente in cod_matriculas.values():
    gera_relatorio(df_componente, lista_componentes)

In [17]:
len(lista_componentes)

287

In [18]:
def exibe_relatorio(lista_componentes):
    for componente in lista_componentes:
        print('Disciplina: {}'.format(componente.get_nome_componente()))
        print('Código: {}\n'.format(componente.get_cod()))
        print('Qtd matriculas: {}\n'.format(componente.get_qtd_matriculas()))
        print('Aprovacoes: {}'.format(componente.get_aprovacoes()))
        print('% aprovacoes: {}\n'.format(componente.get_porcentagem_aprovacao()))
        print('Reprovacoes: {}'.format(componente.get_reprovacoes()))
        print('% reprovacoes: {}\n'.format(componente.get_porcentagem_reprovacao()))
        print('{}'.format(componente.get_indices_str()))
        #print()
        
exibe_relatorio(lista_componentes)

Disciplina: COMPLEMENTOS EM FISICA GERAL I
Código: 1101101

Qtd matriculas: 9

Aprovacoes: 9
% aprovacoes: 1.0

Reprovacoes: 0
% reprovacoes: 0.0

APROVADO: 9
% APROVADO: 1.0
DISPENSADO: 0
% DISPENSADO: 0.0
TRANCADO: 0
% TRANCADO: 0.0
REPROVADO: 0
% REPROVADO: 0.0
REP. FALTA: 0
% REP. FALTA: 0.0
INDEFERIDO: 0
% INDEFERIDO: 0.0
EXCLUIDA: 0
% EXCLUIDA: 0.0
DESISTENCIA: 0
% DESISTENCIA: 0.0
MATRICULADO: 0
% MATRICULADO: 0.0
CANCELADO: 0
% CANCELADO: 0.0
APROVEITADO: 0
% APROVEITADO: 0.0

Disciplina: ESTRUTURA DA MATÉRIA I
Código: 1101107

Qtd matriculas: 1

Aprovacoes: 1
% aprovacoes: 1.0

Reprovacoes: 0
% reprovacoes: 0.0

APROVADO: 1
% APROVADO: 1.0
DISPENSADO: 0
% DISPENSADO: 0.0
TRANCADO: 0
% TRANCADO: 0.0
REPROVADO: 0
% REPROVADO: 0.0
REP. FALTA: 0
% REP. FALTA: 0.0
INDEFERIDO: 0
% INDEFERIDO: 0.0
EXCLUIDA: 0
% EXCLUIDA: 0.0
DESISTENCIA: 0
% DESISTENCIA: 0.0
MATRICULADO: 0
% MATRICULADO: 0.0
CANCELADO: 0
% CANCELADO: 0.0
APROVEITADO: 0
% APROVEITADO: 0.0

Disciplina: FISICA GERAL I
C

% reprovacoes: 0.005970149253731343

APROVADO: 251
% APROVADO: 0.7492537313432835
DISPENSADO: 44
% DISPENSADO: 0.13134328358208955
TRANCADO: 21
% TRANCADO: 0.0626865671641791
REPROVADO: 2
% REPROVADO: 0.005970149253731343
REP. FALTA: 10
% REP. FALTA: 0.029850746268656716
INDEFERIDO: 4
% INDEFERIDO: 0.011940298507462687
EXCLUIDA: 0
% EXCLUIDA: 0.0
DESISTENCIA: 2
% DESISTENCIA: 0.005970149253731343
MATRICULADO: 0
% MATRICULADO: 0.0
CANCELADO: 1
% CANCELADO: 0.0029850746268656717
APROVEITADO: 0
% APROVEITADO: 0.0

Disciplina: MET TEC PESQ I C SOC
Código: 1401137

Qtd matriculas: 1

Aprovacoes: 0
% aprovacoes: 0.0

Reprovacoes: 0
% reprovacoes: 0.0

APROVADO: 0
% APROVADO: 0.0
DISPENSADO: 1
% DISPENSADO: 1.0
TRANCADO: 0
% TRANCADO: 0.0
REPROVADO: 0
% REPROVADO: 0.0
REP. FALTA: 0
% REP. FALTA: 0.0
INDEFERIDO: 0
% INDEFERIDO: 0.0
EXCLUIDA: 0
% EXCLUIDA: 0.0
DESISTENCIA: 0
% DESISTENCIA: 0.0
MATRICULADO: 0
% MATRICULADO: 0.0
CANCELADO: 0
% CANCELADO: 0.0
APROVEITADO: 0
% APROVEITADO: 0.0

Dis

In [19]:
#disciplinas ordenadas pela taxa de aprovacao

lista_componentes.sort(key=lambda x: x.porcentagem_aprovacao, reverse=True)

exibe_relatorio(lista_componentes)

Disciplina: COMPLEMENTOS EM FISICA GERAL I
Código: 1101101

Qtd matriculas: 9

Aprovacoes: 9
% aprovacoes: 1.0

Reprovacoes: 0
% reprovacoes: 0.0

APROVADO: 9
% APROVADO: 1.0
DISPENSADO: 0
% DISPENSADO: 0.0
TRANCADO: 0
% TRANCADO: 0.0
REPROVADO: 0
% REPROVADO: 0.0
REP. FALTA: 0
% REP. FALTA: 0.0
INDEFERIDO: 0
% INDEFERIDO: 0.0
EXCLUIDA: 0
% EXCLUIDA: 0.0
DESISTENCIA: 0
% DESISTENCIA: 0.0
MATRICULADO: 0
% MATRICULADO: 0.0
CANCELADO: 0
% CANCELADO: 0.0
APROVEITADO: 0
% APROVEITADO: 0.0

Disciplina: ESTRUTURA DA MATÉRIA I
Código: 1101107

Qtd matriculas: 1

Aprovacoes: 1
% aprovacoes: 1.0

Reprovacoes: 0
% reprovacoes: 0.0

APROVADO: 1
% APROVADO: 1.0
DISPENSADO: 0
% DISPENSADO: 0.0
TRANCADO: 0
% TRANCADO: 0.0
REPROVADO: 0
% REPROVADO: 0.0
REP. FALTA: 0
% REP. FALTA: 0.0
INDEFERIDO: 0
% INDEFERIDO: 0.0
EXCLUIDA: 0
% EXCLUIDA: 0.0
DESISTENCIA: 0
% DESISTENCIA: 0.0
MATRICULADO: 0
% MATRICULADO: 0.0
CANCELADO: 0
% CANCELADO: 0.0
APROVEITADO: 0
% APROVEITADO: 0.0

Disciplina: MECANICA GERAL I

Reprovacoes: 0
% reprovacoes: 0.0

APROVADO: 46
% APROVADO: 0.7540983606557377
DISPENSADO: 1
% DISPENSADO: 0.01639344262295082
TRANCADO: 12
% TRANCADO: 0.19672131147540983
REPROVADO: 0
% REPROVADO: 0.0
REP. FALTA: 2
% REP. FALTA: 0.03278688524590164
INDEFERIDO: 0
% INDEFERIDO: 0.0
EXCLUIDA: 0
% EXCLUIDA: 0.0
DESISTENCIA: 0
% DESISTENCIA: 0.0
MATRICULADO: 0
% MATRICULADO: 0.0
CANCELADO: 0
% CANCELADO: 0.0
APROVEITADO: 0
% APROVEITADO: 0.0

Disciplina: INTRODUCAO A SOCIOLOGIA
Código: 1401134

Qtd matriculas: 335

Aprovacoes: 251
% aprovacoes: 0.7492537313432835

Reprovacoes: 2
% reprovacoes: 0.005970149253731343

APROVADO: 251
% APROVADO: 0.7492537313432835
DISPENSADO: 44
% DISPENSADO: 0.13134328358208955
TRANCADO: 21
% TRANCADO: 0.0626865671641791
REPROVADO: 2
% REPROVADO: 0.005970149253731343
REP. FALTA: 10
% REP. FALTA: 0.029850746268656716
INDEFERIDO: 4
% INDEFERIDO: 0.011940298507462687
EXCLUIDA: 0
% EXCLUIDA: 0.0
DESISTENCIA: 2
% DESISTENCIA: 0.005970149253731343
MATRICULADO: 0
% M

Disciplina: INTRODUCAO A ALGEBRA LINEAR
Código: 1103179

Qtd matriculas: 213

Aprovacoes: 132
% aprovacoes: 0.6197183098591549

Reprovacoes: 38
% reprovacoes: 0.1784037558685446

APROVADO: 132
% APROVADO: 0.6197183098591549
DISPENSADO: 29
% DISPENSADO: 0.13615023474178403
TRANCADO: 2
% TRANCADO: 0.009389671361502348
REPROVADO: 38
% REPROVADO: 0.1784037558685446
REP. FALTA: 11
% REP. FALTA: 0.051643192488262914
INDEFERIDO: 1
% INDEFERIDO: 0.004694835680751174
EXCLUIDA: 0
% EXCLUIDA: 0.0
DESISTENCIA: 0
% DESISTENCIA: 0.0
MATRICULADO: 0
% MATRICULADO: 0.0
CANCELADO: 0
% CANCELADO: 0.0
APROVEITADO: 0
% APROVEITADO: 0.0

Disciplina: FISICA APLICADA A COMPUTACAO I
Código: 1101171

Qtd matriculas: 219

Aprovacoes: 135
% aprovacoes: 0.6164383561643836

Reprovacoes: 28
% reprovacoes: 0.1278538812785388

APROVADO: 135
% APROVADO: 0.6164383561643836
DISPENSADO: 22
% DISPENSADO: 0.1004566210045662
TRANCADO: 6
% TRANCADO: 0.0273972602739726
REPROVADO: 28
% REPROVADO: 0.1278538812785388
REP. FALTA: 

Disciplina: ENGENHARIA ALIM SOCIEDADE
Código: 1702107

Qtd matriculas: 1

Aprovacoes: 0
% aprovacoes: 0.0

Reprovacoes: 0
% reprovacoes: 0.0

APROVADO: 0
% APROVADO: 0.0
DISPENSADO: 1
% DISPENSADO: 1.0
TRANCADO: 0
% TRANCADO: 0.0
REPROVADO: 0
% REPROVADO: 0.0
REP. FALTA: 0
% REP. FALTA: 0.0
INDEFERIDO: 0
% INDEFERIDO: 0.0
EXCLUIDA: 0
% EXCLUIDA: 0.0
DESISTENCIA: 0
% DESISTENCIA: 0.0
MATRICULADO: 0
% MATRICULADO: 0.0
CANCELADO: 0
% CANCELADO: 0.0
APROVEITADO: 0
% APROVEITADO: 0.0

Disciplina: MICROBIOLOGIA DE ALIMENI
Código: 1702127

Qtd matriculas: 1

Aprovacoes: 0
% aprovacoes: 0.0

Reprovacoes: 0
% reprovacoes: 0.0

APROVADO: 0
% APROVADO: 0.0
DISPENSADO: 0
% DISPENSADO: 0.0
TRANCADO: 1
% TRANCADO: 1.0
REPROVADO: 0
% REPROVADO: 0.0
REP. FALTA: 0
% REP. FALTA: 0.0
INDEFERIDO: 0
% INDEFERIDO: 0.0
EXCLUIDA: 0
% EXCLUIDA: 0.0
DESISTENCIA: 0
% DESISTENCIA: 0.0
MATRICULADO: 0
% MATRICULADO: 0.0
CANCELADO: 0
% CANCELADO: 0.0
APROVEITADO: 0
% APROVEITADO: 0.0

Disciplina: MECANICA DOS FLUIDO

In [20]:
#disciplinas ordenadas pela taxa de reprovacao

lista_componentes.sort(key=lambda x: x.porcentagem_reprovacao, reverse=True)

exibe_relatorio(lista_componentes)

Disciplina: ALGEBRA LINEAR
Código: 1103105

Qtd matriculas: 2

Aprovacoes: 1
% aprovacoes: 0.5

Reprovacoes: 1
% reprovacoes: 0.5

APROVADO: 1
% APROVADO: 0.5
DISPENSADO: 0
% DISPENSADO: 0.0
TRANCADO: 0
% TRANCADO: 0.0
REPROVADO: 1
% REPROVADO: 0.5
REP. FALTA: 0
% REP. FALTA: 0.0
INDEFERIDO: 0
% INDEFERIDO: 0.0
EXCLUIDA: 0
% EXCLUIDA: 0.0
DESISTENCIA: 0
% DESISTENCIA: 0.0
MATRICULADO: 0
% MATRICULADO: 0.0
CANCELADO: 0
% CANCELADO: 0.0
APROVEITADO: 0
% APROVEITADO: 0.0

Disciplina: FUNCOES DE UMA VARIAVEL COMPLEXA
Código: 1103128

Qtd matriculas: 2

Aprovacoes: 0
% aprovacoes: 0.0

Reprovacoes: 1
% reprovacoes: 0.5

APROVADO: 0
% APROVADO: 0.0
DISPENSADO: 0
% DISPENSADO: 0.0
TRANCADO: 0
% TRANCADO: 0.0
REPROVADO: 1
% REPROVADO: 0.5
REP. FALTA: 1
% REP. FALTA: 0.5
INDEFERIDO: 0
% INDEFERIDO: 0.0
EXCLUIDA: 0
% EXCLUIDA: 0.0
DESISTENCIA: 0
% DESISTENCIA: 0.0
MATRICULADO: 0
% MATRICULADO: 0.0
CANCELADO: 0
% CANCELADO: 0.0
APROVEITADO: 0
% APROVEITADO: 0.0

Disciplina: MECANICA DOS FLUIDOS
C

% reprovacoes: 0.0026954177897574125

APROVADO: 282
% APROVADO: 0.7601078167115903
DISPENSADO: 29
% DISPENSADO: 0.07816711590296496
TRANCADO: 6
% TRANCADO: 0.016172506738544475
REPROVADO: 1
% REPROVADO: 0.0026954177897574125
REP. FALTA: 6
% REP. FALTA: 0.016172506738544475
INDEFERIDO: 44
% INDEFERIDO: 0.11859838274932614
EXCLUIDA: 3
% EXCLUIDA: 0.008086253369272238
DESISTENCIA: 0
% DESISTENCIA: 0.0
MATRICULADO: 0
% MATRICULADO: 0.0
CANCELADO: 0
% CANCELADO: 0.0
APROVEITADO: 0
% APROVEITADO: 0.0

Disciplina: INTRODUCAO A ADMINISTRACAO
Código: 1204140

Qtd matriculas: 494

Aprovacoes: 445
% aprovacoes: 0.9008097165991903

Reprovacoes: 1
% reprovacoes: 0.0020242914979757085

APROVADO: 445
% APROVADO: 0.9008097165991903
DISPENSADO: 30
% DISPENSADO: 0.06072874493927125
TRANCADO: 10
% TRANCADO: 0.020242914979757085
REPROVADO: 1
% REPROVADO: 0.0020242914979757085
REP. FALTA: 8
% REP. FALTA: 0.016194331983805668
INDEFERIDO: 0
% INDEFERIDO: 0.0
EXCLUIDA: 0
% EXCLUIDA: 0.0
DESISTENCIA: 0
% DESIS

% aprovacoes: 0.816

Reprovacoes: 0
% reprovacoes: 0.0

APROVADO: 204
% APROVADO: 0.816
DISPENSADO: 31
% DISPENSADO: 0.124
TRANCADO: 8
% TRANCADO: 0.032
REPROVADO: 0
% REPROVADO: 0.0
REP. FALTA: 3
% REP. FALTA: 0.012
INDEFERIDO: 3
% INDEFERIDO: 0.012
EXCLUIDA: 0
% EXCLUIDA: 0.0
DESISTENCIA: 1
% DESISTENCIA: 0.004
MATRICULADO: 0
% MATRICULADO: 0.0
CANCELADO: 0
% CANCELADO: 0.0
APROVEITADO: 0
% APROVEITADO: 0.0

Disciplina: TRABALHO INDIVIDUAL EM BANCO DE DADOS
Código: 1107227

Qtd matriculas: 16

Aprovacoes: 13
% aprovacoes: 0.8125

Reprovacoes: 0
% reprovacoes: 0.0

APROVADO: 13
% APROVADO: 0.8125
DISPENSADO: 0
% DISPENSADO: 0.0
TRANCADO: 0
% TRANCADO: 0.0
REPROVADO: 0
% REPROVADO: 0.0
REP. FALTA: 1
% REP. FALTA: 0.0625
INDEFERIDO: 1
% INDEFERIDO: 0.0625
EXCLUIDA: 0
% EXCLUIDA: 0.0
DESISTENCIA: 1
% DESISTENCIA: 0.0625
MATRICULADO: 0
% MATRICULADO: 0.0
CANCELADO: 0
% CANCELADO: 0.0
APROVEITADO: 0
% APROVEITADO: 0.0

Disciplina: PERIFERICOS E INTERFACE
Código: 1107158

Qtd matriculas: 62

Qtd matriculas: 2

Aprovacoes: 1
% aprovacoes: 0.5

Reprovacoes: 0
% reprovacoes: 0.0

APROVADO: 1
% APROVADO: 0.5
DISPENSADO: 0
% DISPENSADO: 0.0
TRANCADO: 1
% TRANCADO: 0.5
REPROVADO: 0
% REPROVADO: 0.0
REP. FALTA: 0
% REP. FALTA: 0.0
INDEFERIDO: 0
% INDEFERIDO: 0.0
EXCLUIDA: 0
% EXCLUIDA: 0.0
DESISTENCIA: 0
% DESISTENCIA: 0.0
MATRICULADO: 0
% MATRICULADO: 0.0
CANCELADO: 0
% CANCELADO: 0.0
APROVEITADO: 0
% APROVEITADO: 0.0

Disciplina: MECANICA GERAL
Código: 1701142

Qtd matriculas: 2

Aprovacoes: 1
% aprovacoes: 0.5

Reprovacoes: 0
% reprovacoes: 0.0

APROVADO: 1
% APROVADO: 0.5
DISPENSADO: 1
% DISPENSADO: 0.5
TRANCADO: 0
% TRANCADO: 0.0
REPROVADO: 0
% REPROVADO: 0.0
REP. FALTA: 0
% REP. FALTA: 0.0
INDEFERIDO: 0
% INDEFERIDO: 0.0
EXCLUIDA: 0
% EXCLUIDA: 0.0
DESISTENCIA: 0
% DESISTENCIA: 0.0
MATRICULADO: 0
% MATRICULADO: 0.0
CANCELADO: 0
% CANCELADO: 0.0
APROVEITADO: 0
% APROVEITADO: 0.0

Disciplina: METROLOGIA INDUSTRIAL
Código: 1701143

Qtd matriculas: 2

Aprovacoes: 1
% aprovacoes

% aprovacoes: 0.0

Reprovacoes: 0
% reprovacoes: 0.0

APROVADO: 0
% APROVADO: 0.0
DISPENSADO: 1
% DISPENSADO: 1.0
TRANCADO: 0
% TRANCADO: 0.0
REPROVADO: 0
% REPROVADO: 0.0
REP. FALTA: 0
% REP. FALTA: 0.0
INDEFERIDO: 0
% INDEFERIDO: 0.0
EXCLUIDA: 0
% EXCLUIDA: 0.0
DESISTENCIA: 0
% DESISTENCIA: 0.0
MATRICULADO: 0
% MATRICULADO: 0.0
CANCELADO: 0
% CANCELADO: 0.0
APROVEITADO: 0
% APROVEITADO: 0.0

Disciplina: INTRODUCAO AO PROJETO
Código: 1701129

Qtd matriculas: 1

Aprovacoes: 0
% aprovacoes: 0.0

Reprovacoes: 0
% reprovacoes: 0.0

APROVADO: 0
% APROVADO: 0.0
DISPENSADO: 0
% DISPENSADO: 0.0
TRANCADO: 1
% TRANCADO: 1.0
REPROVADO: 0
% REPROVADO: 0.0
REP. FALTA: 0
% REP. FALTA: 0.0
INDEFERIDO: 0
% INDEFERIDO: 0.0
EXCLUIDA: 0
% EXCLUIDA: 0.0
DESISTENCIA: 0
% DESISTENCIA: 0.0
MATRICULADO: 0
% MATRICULADO: 0.0
CANCELADO: 0
% CANCELADO: 0.0
APROVEITADO: 0
% APROVEITADO: 0.0

Disciplina: ACONDICIONAMENTO E EMBALAGEM
Código: 1702100

Qtd matriculas: 1

Aprovacoes: 0
% aprovacoes: 0.0

Reprovacoes: